In [1]:
# from pycaret.time_series import *

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import category_encoders as ce

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
# import plotly.express as px
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import KFold

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
# from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
# from sklearn.base import ClassifierMixin

# CatBoost
# from catboost import CatBoostRegressor

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
from torch.utils.data import DataLoader

# for Torch hyper parameter tuning
from functools import partial
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
# from ray import tune
# from ray.tune import CLIReporter
# from ray.tune.schedulers import ASHAScheduler

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

pd.set_option('display.max_columns', None)

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [4]:
path = '../data/daegu/automl/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

display(train.head())
display(test.head())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차


In [5]:
print(train.columns)
print(test.columns)

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',
       '경상자수', '부상자수', 'ECLO'],
      dtype='object')
Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형'], dtype='object')


In [6]:
# train_df = train.drop(['ID'],axis=1)
# test_df = test.drop(['ID'],axis=1)

train_df = train
test_df = test


In [7]:
train_df.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3


## train, test 데이터 기간 확인하기

In [8]:
display(f"train : {train_df.iloc[0]['사고일시']} ~ {train_df.iloc[-1]['사고일시']}")
display(f"test : {test_df.iloc[0]['사고일시']} ~ {test_df.iloc[-1]['사고일시']}")

'train : 2019-01-01 00 ~ 2021-12-31 23'

'test : 2022-01-01 01 ~ 2022-12-31 21'

## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [9]:
time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
# train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
# test_df = test_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',\n       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',\n       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',\n       '경상자수', '부상자수', 'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월',\n       '일', '시간'],\n      dtype='object')"

## **파생 변수 생성 2 : 공간(위치) 정보 생성**

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [10]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간',\n       '도시', '구', '동'],\n      dtype='object')"

## **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [11]:
road_pattern = r'(.+) - (.+)'

# train_df[['도로형태1', '도로형태2']] = train['도로형태'].str.extract(road_pattern)
# train_df = train_df.drop(columns=['도로형태'])

# test_df[['도로형태1', '도로형태2']] = test['도로형태'].str.extract(road_pattern)
# test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간',\n       '도시', '구', '동'],\n      dtype='object')"

## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다

In [12]:
display(train_df.head())
display(test_df.head())

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,도시,구,동
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,2019,1,1,0,대구광역시,중구,대신동
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,2019,1,1,0,대구광역시,달서구,감삼동
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3,2019,1,1,1,대구광역시,수성구,두산동
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5,2019,1,1,2,대구광역시,북구,복현동
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3,2019,1,1,4,대구광역시,동구,신암동


,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,연,월,일,시간,도시,구,동
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,교차로 - 교차로안,건조,차대사람,2022,1,1,1,대구광역시,수성구,상동
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,단일로 - 기타,건조,차대사람,2022,1,1,1,대구광역시,수성구,지산동
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,교차로 - 교차로안,건조,차대차,2022,1,1,4,대구광역시,수성구,수성동2가
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,단일로 - 기타,건조,차대차,2022,1,1,4,대구광역시,수성구,신매동
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,교차로 - 교차로안,건조,차대차,2022,1,1,6,대구광역시,달서구,감삼동


In [13]:
train_df.drop('도시',axis=1,inplace=True)
test_df.drop('도시',axis=1,inplace=True)

## 추가 데이터 불러오기

- 대구 보안등 정보
- 대구 어린이 보호 구역 정보
- 대구 주차장 정보
- 대구 CCTV 정보
- countrywide_accident

In [14]:
light = pd.read_csv(path+'대구 보안등 정보.csv',encoding='cp949')
light.head()

,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치연도,설치형태
0,대명1동1,1,대구광역시 남구 현충로 155,대구광역시 남구 대명동 1722-4,35.846703,128.579803,2016.0,한전주
1,대명1동2,1,대구광역시 남구 현충로31길 10-1,대구광역시 남구 대명동 1723-11,35.846863,128.579406,2016.0,한전주
2,대명1동3,1,대구광역시 남구 현충로31길 9-8,대구광역시 남구 대명동 1722-56,35.846341,128.579342,2017.0,건축물
3,대명1동4,2,대구광역시 남구 현충로31길 19-11,대구광역시 남구 대명동 1722-41,35.846368,128.578922,2016.0,한전주
4,대명1동5,1,대구광역시 남구 현충로29길 22-2,대구광역시 남구 대명동 1721-22,35.845995,128.578858,2016.0,한전주


In [15]:
def black_split_1(x):
    return x.split(' ')[1]

In [16]:
def black_split_2(x):
    try:
        return x.split(' ')[2]
    except:
        return x

In [17]:
light['구'] = light['소재지지번주소'].apply(black_split_1)
light['동'] = light['소재지지번주소'].apply(black_split_2)
light.head()

,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치연도,설치형태,구,동
0,대명1동1,1,대구광역시 남구 현충로 155,대구광역시 남구 대명동 1722-4,35.846703,128.579803,2016.0,한전주,남구,대명동
1,대명1동2,1,대구광역시 남구 현충로31길 10-1,대구광역시 남구 대명동 1723-11,35.846863,128.579406,2016.0,한전주,남구,대명동
2,대명1동3,1,대구광역시 남구 현충로31길 9-8,대구광역시 남구 대명동 1722-56,35.846341,128.579342,2017.0,건축물,남구,대명동
3,대명1동4,2,대구광역시 남구 현충로31길 19-11,대구광역시 남구 대명동 1722-41,35.846368,128.578922,2016.0,한전주,남구,대명동
4,대명1동5,1,대구광역시 남구 현충로29길 22-2,대구광역시 남구 대명동 1721-22,35.845995,128.578858,2016.0,한전주,남구,대명동


In [18]:
pattern = r'(\w+동)'
light['동'] = light['동'].str.extract(pattern)

In [19]:
# dong_install = light.groupby(['동'])[['설치개수']].mean()
light.rename(columns={'설치개수':'보안등개수'},inplace = True)
dong_install = light.groupby(['동'])[['보안등개수']].sum() #📢sum으로 바꿈
dong_install

,보안등개수
동,
3가동,491
3동,1636
각산동,139
갈산동,351
감삼동,941
...,...
호산동,214
화전동,22
황금1동,153


In [20]:
# dong_install.to_csv('../data/daegu/automl/동별_보안등설치개수.csv',encoding='utf-8')

In [22]:
pd.read_csv(path+'동별_보안등개수.csv')

,구,동,어린이보호구역개수
0,남구,대명동,26
1,남구,봉덕동,8
2,남구,이천동,6
3,달성군,가창면,4
4,달성군,구지면,3
...,...,...,...
66,중구,봉산동,2
67,중구,삼덕동2가,1
68,중구,삼덕동3가,1
69,중구,서문로1가,1


In [23]:
chil_info = pd.read_csv(path+'대구 어린이 보호 구역 정보.csv',encoding='cp949')
chil_info.head()

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자
0,초등학교,남도초등학교,대구광역시 남구 현충동길 74(대명동),대구광역시 남구 대명동 1709,35.845027,128.581402,대구광역시,남부경찰서,Y,7.0,6~8,2020-03-23
1,초등학교,영선초등학교,대구광역시 남구 영선길96(이천동),대구광역시 남구 이천동 477,35.852901,128.596014,대구광역시,남부경찰서,Y,8.0,6~10,2020-03-23
2,초등학교,성명초등학교,대구광역시 남구 성당로 30길 55(대명동),대구광역시 남구 대명동 3050,35.845152,128.570825,대구광역시,남부경찰서,Y,14.0,8~12,2020-03-23
3,초등학교,남덕초등학교,대구광역시 남구 앞산순환로 93길 33,대구광역시 남구 대명동 531-1,35.833042,128.573949,대구광역시,남부경찰서,Y,6.0,6~8,2020-03-23
4,초등학교,대명초등학교,대구광역시 남구 대명로 110,대구광역시 남구 대명동 960,35.838869,128.568889,대구광역시,남부경찰서,Y,5.0,6~10,2020-03-23


In [24]:
def black_split_1(x):
    try:
        return x.split(' ')[1]
    except:
        return x

In [25]:
def black_split_2(x):
    try:
        return x.split(' ')[2]
    except:
        return x

In [26]:
chil_info['구'] = chil_info['소재지지번주소'].apply(black_split_1)
chil_info['동'] = chil_info['소재지지번주소'].apply(black_split_2)

chil_info.head()

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,구,동
0,초등학교,남도초등학교,대구광역시 남구 현충동길 74(대명동),대구광역시 남구 대명동 1709,35.845027,128.581402,대구광역시,남부경찰서,Y,7.0,6~8,2020-03-23,남구,대명동
1,초등학교,영선초등학교,대구광역시 남구 영선길96(이천동),대구광역시 남구 이천동 477,35.852901,128.596014,대구광역시,남부경찰서,Y,8.0,6~10,2020-03-23,남구,이천동
2,초등학교,성명초등학교,대구광역시 남구 성당로 30길 55(대명동),대구광역시 남구 대명동 3050,35.845152,128.570825,대구광역시,남부경찰서,Y,14.0,8~12,2020-03-23,남구,대명동
3,초등학교,남덕초등학교,대구광역시 남구 앞산순환로 93길 33,대구광역시 남구 대명동 531-1,35.833042,128.573949,대구광역시,남부경찰서,Y,6.0,6~8,2020-03-23,남구,대명동
4,초등학교,대명초등학교,대구광역시 남구 대명로 110,대구광역시 남구 대명동 960,35.838869,128.568889,대구광역시,남부경찰서,Y,5.0,6~10,2020-03-23,남구,대명동


In [27]:
pattern = r'(\w+동)'
chil_info['동'] = chil_info['동'].str.extract(pattern)

In [28]:
# cctv_count = chil_info[chil_info['CCTV설치대수'].notnull()].groupby(['동'])[['CCTV설치대수']].mean()
cctv_count = chil_info[chil_info['CCTV설치대수'].notnull()].groupby(['동'])[['CCTV설치대수']].sum() #📢sum으로 바꿈
cctv_count

,CCTV설치대수
동,
검단동,4.0
고성동,4.0
관음동,22.0
구암동,36.0
국우동,16.0
남산동,61.0
남일동,1.0
노원동,6.0
대명동,106.0


In [29]:
parking_df = pd.read_csv(path+'대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])
parking_df = parking_df.groupby(['도시', '구', '동']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [30]:
parking_df.head(3)

,도시,구,동,급지구분_1,급지구분_2,급지구분_3
0,대구광역시,남구,대명동,20,1,0
1,대구광역시,남구,봉덕동,9,3,0
2,대구광역시,남구,이천동,3,0,0


In [31]:
pd.read_csv(path+'주차장급지구분별개수.csv')

,구,동,급지구분_1,급지구분_2,급지구분_3
0,남구,대명동,20,1,0
1,남구,봉덕동,9,3,0
2,남구,이천동,3,0,0
3,달서구,갈산동,0,0,4
4,달서구,감삼동,0,1,3
...,...,...,...,...,...
131,중구,태평로2가,5,0,0
132,중구,태평로3가,1,0,0
133,중구,포정동,4,0,0
134,중구,향촌동,2,0,0


In [32]:
cctv = pd.read_csv(path+'대구 CCTV 정보.csv',encoding='cp949')
cctv.head()

,무인교통단속카메라관리번호,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,설치장소,단속구분,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치연도
0,주차12,대구광역시,남구,시도,NaN,대명로,3,대구광역시 남구 대명로 162,대구광역시 남구 대명동 903-13,35.838857,128.575418,안지랑네거리1 버스정류장,4,0,NaN,NaN,99.0,2010.0
1,주차13,대구광역시,남구,시도,NaN,봉덕로,3,대구광역시 남구 봉덕로 135,대구광역시 남구 봉덕동 853-8,35.844653,128.604619,효성백년가약,4,0,NaN,NaN,99.0,2013.0
2,주차14,대구광역시,남구,시도,NaN,대봉로,3,대구광역시 남구 봉덕로 134,대구광역시 남구 봉덕동 866-46,35.844183,128.604815,강변코오롱하늘채,4,0,NaN,NaN,99.0,2013.0
3,주차15,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 200,대구광역시 남구 대명동 2680-17,35.834798,128.585284,보성청록타운,4,0,NaN,NaN,99.0,2014.0
4,주차16,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 243,대구광역시 남구 봉덕동 1616,35.832728,128.588324,환경차고지,4,0,NaN,NaN,99.0,2014.0


In [33]:
cctv['구'] = cctv['소재지지번주소'].apply(black_split_1)
cctv['동'] = cctv['소재지지번주소'].apply(black_split_2)
cctv.head()

,무인교통단속카메라관리번호,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,설치장소,단속구분,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치연도,구,동
0,주차12,대구광역시,남구,시도,NaN,대명로,3,대구광역시 남구 대명로 162,대구광역시 남구 대명동 903-13,35.838857,128.575418,안지랑네거리1 버스정류장,4,0,NaN,NaN,99.0,2010.0,남구,대명동
1,주차13,대구광역시,남구,시도,NaN,봉덕로,3,대구광역시 남구 봉덕로 135,대구광역시 남구 봉덕동 853-8,35.844653,128.604619,효성백년가약,4,0,NaN,NaN,99.0,2013.0,남구,봉덕동
2,주차14,대구광역시,남구,시도,NaN,대봉로,3,대구광역시 남구 봉덕로 134,대구광역시 남구 봉덕동 866-46,35.844183,128.604815,강변코오롱하늘채,4,0,NaN,NaN,99.0,2013.0,남구,봉덕동
3,주차15,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 200,대구광역시 남구 대명동 2680-17,35.834798,128.585284,보성청록타운,4,0,NaN,NaN,99.0,2014.0,남구,대명동
4,주차16,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 243,대구광역시 남구 봉덕동 1616,35.832728,128.588324,환경차고지,4,0,NaN,NaN,99.0,2014.0,남구,봉덕동


In [34]:
speed_cctv = cctv[cctv['제한속도'] > 0].groupby(['동'])[['제한속도']].mean()
speed_cctv

,제한속도
동,
가창면,54.117647
가천동,60.000000
각산동,40.000000
갈산동,50.000000
감삼동,52.500000
...,...
호산동,45.000000
화원읍,50.512821
황금동,52.222222


In [35]:
# speed_cctv.to_csv(path+'동별_제한속도.csv')
pd.read_csv(path+'동별_제한속도.csv')

,동,제한속도
0,가창면,54.117647
1,가천동,60.000000
2,각산동,40.000000
3,갈산동,50.000000
4,감삼동,52.500000
...,...,...
153,호산동,45.000000
154,화원읍,50.512821
155,황금동,52.222222
156,효령면,60.000000


In [86]:
countrywide = pd.read_csv(path+'external_open/countrywide_accident.csv')
countrywide.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/daegu/automl/external_open/countrywide_accident.csv'

In [ ]:
countrywide_df = countrywide.drop(['ID'],axis=1)
countrywide_df.head()

### 데이터 합치기

- dong_install
- cctv_count
- speed_cctv
- parking_df

In [ ]:
# train_df.tail()

In [93]:
train_df.columns

Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',
       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',
       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',
       'ECLO', '연', '월', '일', '시간', '구', '동'],
      dtype='object')

In [94]:
dong_install = dong_install.reset_index()
dong_install.head()

,동,보안등개수
0,3가동,491
1,3동,1636
2,각산동,139
3,갈산동,351
4,감삼동,941


In [36]:
cctv_count = cctv_count.reset_index()
cctv_count.head()

,동,CCTV설치대수
0,검단동,4.0
1,고성동,4.0
2,관음동,22.0
3,구암동,36.0
4,국우동,16.0


In [37]:
speed_cctv  = speed_cctv
speed_cctv.head()

,제한속도
동,
가창면,54.117647
가천동,60.000000
각산동,40.000000
갈산동,50.000000
감삼동,52.500000


In [38]:
train_df.shape

(39609, 28)

In [39]:
tr_df = pd.merge(train_df,dong_install,how='left',on='동')
tr_df.head()

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,구,동,보안등개수
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,2019,1,1,0,중구,대신동,391.0
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,2019,1,1,0,달서구,감삼동,941.0
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3,2019,1,1,1,수성구,두산동,473.0
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5,2019,1,1,2,북구,복현동,534.0
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3,2019,1,1,4,동구,신암동,2064.0


In [40]:
tr_df = pd.merge(tr_df,cctv_count,how='left',on='동')
tr_df = pd.merge(tr_df,speed_cctv,how='left',on='동')
tr_df = pd.merge(tr_df,parking_df.drop(columns='도시'),how='left',on=['구','동'])
tr_df.head()

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,구,동,보안등개수,CCTV설치대수,제한속도,급지구분_1,급지구분_2,급지구분_3
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,2019,1,1,0,중구,대신동,391.0,13.0,40.0,11.0,0.0,0.0
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,2019,1,1,0,달서구,감삼동,941.0,NaN,52.5,0.0,1.0,3.0
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3,2019,1,1,1,수성구,두산동,473.0,NaN,55.0,NaN,NaN,NaN
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5,2019,1,1,2,북구,복현동,534.0,32.0,38.0,0.0,9.0,5.0
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3,2019,1,1,4,동구,신암동,2064.0,NaN,40.0,0.0,1.0,0.0


In [100]:
test_df.shape

(10963, 13)

In [41]:
te_df = pd.merge(test_df,dong_install,how='left',on='동')
te_df = pd.merge(te_df ,cctv_count,how='left',on='동')
te_df = pd.merge(te_df ,speed_cctv,how='left',on='동')
te_df = pd.merge(te_df, parking_df,how='left',on=['구','동'])
te_df

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,연,월,일,시간,구,동,보안등개수,CCTV설치대수,제한속도,도시,급지구분_1,급지구분_2,급지구분_3
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,교차로 - 교차로안,건조,차대사람,2022,1,1,1,수성구,상동,700.0,NaN,52.500000,NaN,NaN,NaN,NaN
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,단일로 - 기타,건조,차대사람,2022,1,1,1,수성구,지산동,NaN,NaN,47.142857,대구광역시,0.0,0.0,2.0
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,교차로 - 교차로안,건조,차대차,2022,1,1,4,수성구,수성동2가,NaN,NaN,40.000000,NaN,NaN,NaN,NaN
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,단일로 - 기타,건조,차대차,2022,1,1,4,수성구,신매동,NaN,NaN,42.727273,대구광역시,0.0,2.0,1.0
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,교차로 - 교차로안,건조,차대차,2022,1,1,6,달서구,감삼동,941.0,NaN,52.500000,대구광역시,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,단일로 - 터널,건조,차대차,2022,12,31,18,남구,대명동,5377.0,106.0,47.037037,대구광역시,20.0,1.0,0.0
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,단일로 - 기타,건조,차대차,2022,12,31,18,수성구,시지동,NaN,NaN,52.500000,NaN,NaN,NaN,NaN
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,단일로 - 기타,건조,차대차,2022,12,31,20,수성구,연호동,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,교차로 - 교차로부근,건조,차대차,2022,12,31,20,수성구,범물동,NaN,NaN,44.285714,NaN,NaN,NaN,NaN


In [102]:
# 내 데이터 merge 하기
# read_csv
patho = '../data/daegu/'
cross_walk = pd.read_csv(patho+'reverse_geocoding/cross_walk.csv',encoding='cp949')
daegu_geo = pd.read_csv(patho+'reverse_geocoding/daegu_geo.csv',encoding='cp949')
publictraffic = pd.read_csv(patho+'reverse_geocoding/동별_시간별_통행량.csv',encoding='utf-8')

In [103]:
publictraffic.groupby(['동','시간'])['통행량'].sum().reset_index()

,동,시간,통행량
0,가창면,0,1
1,가창면,1,0
2,가창면,2,0
3,가창면,3,0
4,가창면,4,0
...,...,...,...
4027,효목동,19,275421
4028,효목동,20,206502
4029,효목동,21,199679
4030,효목동,22,148972


In [104]:
publictraffic_per_dong_hour = publictraffic.groupby(['동','시간'])['통행량'].sum().reset_index()
display(publictraffic_per_dong_hour.head(3))
cross_walk_per_dong = cross_walk[['동','횡단보도개수']]
cross_walk_per_dong = pd.DataFrame(cross_walk_per_dong.groupby('동')['횡단보도개수'].sum()).reset_index()
display(cross_walk_per_dong.head(3))

print(publictraffic_per_dong_hour.shape, cross_walk_per_dong.shape)

,동,시간,통행량
0,가창면,0,1
1,가창면,1,0
2,가창면,2,0


,동,횡단보도개수
0,가창면,11664
1,가천동,400
2,각산동,45369


(4032, 3) (195, 2)


In [105]:
temp_tr = tr_df.copy()
temp_te = te_df.copy()

In [106]:
# merge
temp_tr = pd.merge(temp_tr, cross_walk_per_dong,how='left',on='동')
temp_tr = pd.merge(temp_tr, publictraffic_per_dong_hour,how='left',on=['동','시간'])
display(temp_tr.head(3))

temp_te = pd.merge(temp_te ,cross_walk_per_dong,how='left',on='동')
temp_te = pd.merge(temp_te ,publictraffic_per_dong_hour,how='left',on=['동','시간'])
display(temp_te.head(3))

print(temp_tr.shape, temp_te.shape)

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,구,동,보안등개수,CCTV설치대수,제한속도,급지구분_1,급지구분_2,급지구분_3,횡단보도개수,통행량
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,2019,1,1,0,중구,대신동,391.0,13.0,40.0,11.0,0.0,0.0,5929.0,40.0
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,2019,1,1,0,달서구,감삼동,941.0,NaN,52.5,0.0,1.0,3.0,10000.0,214.0
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3,2019,1,1,1,수성구,두산동,473.0,NaN,55.0,NaN,NaN,NaN,14641.0,84.0


,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,연,월,일,시간,구,동,보안등개수,CCTV설치대수,제한속도,도시,급지구분_1,급지구분_2,급지구분_3,횡단보도개수,통행량
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,교차로 - 교차로안,건조,차대사람,2022,1,1,1,수성구,상동,700.0,NaN,52.500000,NaN,NaN,NaN,NaN,9025.0,2.0
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,단일로 - 기타,건조,차대사람,2022,1,1,1,수성구,지산동,NaN,NaN,47.142857,대구광역시,0.0,0.0,2.0,52441.0,38.0
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,교차로 - 교차로안,건조,차대차,2022,1,1,4,수성구,수성동2가,NaN,NaN,40.000000,NaN,NaN,NaN,NaN,1764.0,1.0


(39609, 36) (10963, 22)


In [107]:
##################################### 🔻피처 추가
import holidays
def make_holi(x):
    kr_holi = holidays.KR()
    if x in kr_holi:
        # 공휴일vs비공휴일 eclo가 큰 차이를 보이지 않기 때문에 0.5만
        return 0.5
    else:
        return 0

for df in [temp_tr,temp_te]:
    print('?1')
    # 제주감귤 공휴일 추출 코드 참고
    # 공휴일 변수 추가
#     df["공휴일"] = df["사고일시"].map(lambda x : make_holi(x))
    print('?2')
    # 주말 컬럼 추가
    # 주말vs평일 eclo가 큰 차이를 보이지 않기 때문에 0.5만
    df['주말'] = df['요일'].map(lambda x:0.5 if x in ['토요일','일요일'] else 0)
    print('?3')
    # 연평균 eclo의 감소 추세 반영
    # 일단 대충 넣어두기
    years = dict(zip([2019,2020,2021,2022,2023],[4.8,4.7,4.6,4.5,4.4]))
    df['연평균추세']=df['연'].map(lambda x:years.get(x))
    print('?4')
    # 시야감소 컬럼 추가 (시간이 20시~04시 사이인 경우)
    # 이 시간대와 이 시간대 외의 eclo가 0.5~1.4 차이가 나므로 1값을 준다.
    df['시야지수'] = df['시간'].map(lambda x:1 if (x>=20)|(x<=4) else 0)
    print('?5')
    # 주차장 급지구분 일단 주차장 지수로 계산해서 컬럼 수를 줄임
    # 주차장지수 = (1급지 * 4) + (2급지 * 2) + (3급지 * 1)
    df[['급지구분_1','급지구분_2','급지구분_3']] = df[['급지구분_1','급지구분_2','급지구분_3']].fillna(0)
    df['주차장지수'] = df['급지구분_1']*4 + df['급지구분_2']*2 + df['급지구분_3']
    print('?6')



?1
?2
?3
?4
?5
?6
?1
?2
?3
?4
?5
?6


In [108]:
tr_df = temp_tr
te_df = temp_te

In [ ]:
멈춰

In [115]:
cross_walk_per_dong.to_csv(path+'동별_횡단보도개수.csv',index=False)
pd.read_csv(path+'동별_횡단보도개수.csv')

,동,횡단보도개수
0,가창면,11664
1,가천동,400
2,각산동,45369
3,갈산동,6084
4,감삼동,10000
...,...,...
190,호산동,57600
191,화원읍,104976
192,화전동,1
193,황금동,8100


# null 값 처리하기

In [109]:
tr_df.columns, te_df.columns

(Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',
        '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',
        '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',
        'ECLO', '연', '월', '일', '시간', '구', '동', '보안등개수', 'CCTV설치대수', '제한속도',
        '급지구분_1', '급지구분_2', '급지구분_3', '횡단보도개수', '통행량', '주말', '연평균추세', '시야지수',
        '주차장지수'],
       dtype='object'),
 Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간',
        '구', '동', '보안등개수', 'CCTV설치대수', '제한속도', '도시', '급지구분_1', '급지구분_2',
        '급지구분_3', '횡단보도개수', '통행량', '주말', '연평균추세', '시야지수', '주차장지수'],
       dtype='object'))

#### Columns
##### Time feature
    - 사고일시 : year / month(season / quarter / half) / day(월초 / 월말) / hour(morning) / weekday(주말여부)
    
##### Numeric feature
    - 가해운전자 연령 / 피해운전자 연령 / 사망자수 / 중상자수 / 경상자수 / 부상자수
    
##### Categorical feature - high cardinality

## train

In [ ]:
tr_df.info()

In [110]:
tr_df.isnull().sum()

ID                 0
사고일시               0
요일                 0
기상상태               0
도로형태               0
노면상태               0
사고유형               0
사고유형 - 세부분류        0
법규위반               0
가해운전자 차종           0
가해운전자 성별           0
가해운전자 연령           0
가해운전자 상해정도         0
피해운전자 차종         991
피해운전자 성별         991
피해운전자 연령         991
피해운전자 상해정도       991
사망자수               0
중상자수               0
경상자수               0
부상자수               0
ECLO               0
연                  0
월                  0
일                  0
시간                 0
구                  0
동                  0
보안등개수          13740
CCTV설치대수       30837
제한속도            2442
급지구분_1             0
급지구분_2             0
급지구분_3             0
횡단보도개수            62
통행량              300
주말                 0
연평균추세              0
시야지수               0
주차장지수              0
dtype: int64

In [ ]:
#tr_df.drop(tr_df[tr_df['피해운전자 차종'].isnull() & tr_df['피해운전자 성별'].isnull() & tr_df['피해운전자 연령'].isnull() & tr_df['피해운전자 상해정도'].isnull() ].index,inplace=True)
#tr_df

In [ ]:
#tr_df = tr_df.reset_index().iloc[:,1:]

In [ ]:
tr_df.rename(columns={'설치개수':'보안등 설치개수'},inplace=True)

In [ ]:
tr_df.isnull().sum().sort_values(ascending=False)

In [ ]:
corr = tr_df[['CCTV설치대수','보안등 설치개수','제한속도']].corr(numeric_only=True)
corr

In [ ]:
tr_df[['CCTV설치대수','보안등 설치개수','제한속도']].describe()

In [ ]:
tr_df['CCTV설치대수'] = tr_df['CCTV설치대수'].interpolate()
tr_df['보안등 설치개수'] = tr_df['보안등 설치개수'].interpolate()
tr_df['제한속도'] = tr_df['제한속도'].interpolate()
tr_df['횡단보도개수'] = tr_df['횡단보도개수'].interpolate()
tr_df['통행량'] = tr_df['통행량'].interpolate()

In [ ]:
# tr_df.isnull().sum().sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
# warning 표시 무시
import warnings
warnings.filterwarnings('ignore')
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf
# for Jupyter notebook # 한글폰트 설치
import platform

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus']= False

# if platform.system() == 'Darwin': # 맥os 사용자의 경우에
#     rc('font', family = 'AppleGothic')

# elif platform.system() == 'Windows': # 윈도우 사용자의 경우에
#     path = 'c:/Windows/Fonts/malgun.ttf'
#     font_name = font_manager.FontProperties(fname=path).get_name()
#     rc('font', family=font_name)

In [ ]:
# fig, axes = plt.subplots(4,3, figsize=(12,10))

# x_order = tr_df['사고유형'].unique()

# for (ax,k) in zip(axes.flatten(),tr_df['도로형태2'].unique()):
#     sns.countplot(x='사고유형',data=tr_df[tr_df['도로형태2']==k],ax=ax,order=x_order)
#     ax.set_title(f'도로 형태 : {k}')

# plt.tight_layout()
# plt.show()

In [ ]:
# tr_df[tr_df['피해운전자 상해정도'].isnull() & tr_df['피해운전자 연령'].isnull()  & tr_df['피해운전자 성별'].isnull()  & tr_df['피해운전자 차종'].isnull()]['사고유형'].value_counts()

In [ ]:
tr_df['피해운전자 상해정도'].fillna(tr_df['가해운전자 상해정도'],inplace=True)
tr_df['피해운전자 연령'].fillna(tr_df['가해운전자 연령'],inplace=True)
tr_df['피해운전자 성별'].fillna(tr_df['가해운전자 성별'],inplace=True)
tr_df['피해운전자 차종'].fillna(tr_df['가해운전자 차종'],inplace=True)

In [ ]:
# tr_df.isnull().sum().sort_values()

In [ ]:
tr_df.shape

### 시각화

In [ ]:
# tr_df.columns

In [ ]:
# # 요일에 따른 ECLO 변화
# day_ECLO = tr_df.groupby('요일')['ECLO'].mean().sort_values()

# plt.figure(figsize=(8,4))
# plt.title("요일에 따른 ECLO 변화")
# pal = sns.color_palette("Blues_d", len(day_ECLO))

# sns.barplot(x=day_ECLO.index, y=day_ECLO.values, palette=pal)

# plt.xlabel("WEEK")
# plt.ylabel("ECLO")
# plt.tight_layout()

In [ ]:
# # 기상상태에 따른 ECLO 변화
# day_ECLO = tr_df.groupby('기상상태')['ECLO'].mean().sort_values()

# plt.figure(figsize=(8,4))
# plt.title("기상상태에 따른 ECLO 변화")
# pal = sns.color_palette("Blues_d", len(day_ECLO))

# sns.barplot(x=day_ECLO.index, y=day_ECLO.values, palette=pal)

# plt.xlabel("기상상태")
# plt.ylabel("ECLO")
# plt.tight_layout()

In [ ]:
# # 노면상태'에 따른 ECLO 변화
# day_ECLO = tr_df.groupby('노면상태')['ECLO'].mean().sort_values()

# plt.figure(figsize=(8,4))
# plt.title("노면상태'에 따른 ECLO 변화")
# pal = sns.color_palette("Blues_d", len(day_ECLO))

# sns.barplot(x=day_ECLO.index, y=day_ECLO.values, palette=pal)

# plt.xlabel("노면상태")
# plt.ylabel("ECLO")
# plt.tight_layout()

In [ ]:
# # 사고유형 - 세부분류에 따른 ECLO 변화
# day_ECLO = tr_df.groupby('사고유형 - 세부분류')['ECLO'].mean().sort_values()

# plt.figure(figsize=(18,4))
# plt.title("사고유형 - 세부분류에 따른 ECLO 변화")
# pal = sns.color_palette("Blues_d", len(day_ECLO))

# sns.barplot(x=day_ECLO.index, y=day_ECLO.values, palette=pal)

# plt.xlabel("사고유형 - 세부분류")
# plt.ylabel("ECLO")
# plt.tight_layout()

In [ ]:
# Q1 = tr_df['ECLO'].quantile(0.25)
# Q3 = tr_df['ECLO'].quantile(0.75)
# IQR = Q3-Q1

# upper_bound = Q3 + 1.5 * IQR



# plt.figure(figsize=(6,4))
# sns.boxplot(x='ECLO', data=tr_df)

# plt.axvline(x=upper_bound, color='r', linestyle='--', label='Upper Bound')

# plt.text(upper_bound+12, 0.5, f'Upper Bound: {upper_bound:.2f}', color='r', ha='center', va='bottom')

# plt.legend()

# plt.title("box plot of ECLO")
# plt.show()

In [ ]:
high_ECLO_df = tr_df[tr_df['ECLO'] > 10.5]
no_high_ECLO_df = tr_df[tr_df['ECLO'] <= 10.5]

In [ ]:
# # ECLO 10.5 이상은 주말 발생 비율이 높다, 10.5 이하는 주말 사고 발생 비율이 더 낮다.

# x_order = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']

# fig, axes = plt.subplots(1, 2, figsize=(6,4))
# sns.countplot(x='요일', data=high_ECLO_df, order=x_order, ax=axes[0])
# sns.countplot(x='요일', data=no_high_ECLO_df, order=x_order, ax=axes[1])
# axes[0].tick_params(axis='x', rotation=30)
# axes[1].tick_params(axis='x', rotation=30)

# axes[0].set_title("ECLO > 10.5")
# axes[1].set_title("ECLO <= 10.5 ")

# plt.tight_layout()
# plt.show()

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(10,5))
# sns.countplot(x='시간', data=high_ECLO_df,  ax=axes[0])
# sns.countplot(x='시간', data=no_high_ECLO_df,  ax=axes[1])
# axes[0].tick_params(axis='x', rotation=30)
# axes[1].tick_params(axis='x', rotation=30)

# axes[0].set_title("ECLO > 10.5")
# axes[1].set_title("ECLO <= 10.5 ")

# plt.tight_layout()
# plt.show()

In [ ]:
# # 사고 발생 시간대별 ECLO median

# sns.barplot(x='시간', y='ECLO', data=train_df, estimator=np.median, errorbar=None)
# plt.show()

In [ ]:
# # 시간 / 사고형태 / 도로형태1
# plt.figure(figsize=(25, 6))
# sns.barplot(x='시간', y='ECLO', hue='도로형태1', data=tr_df, errorbar=None)
# plt.legend(loc='upper right')
# plt.show()

In [ ]:
# # 시간 / 사고형태 / 도로형태2
# plt.figure(figsize=(25, 6))
# sns.barplot(x='시간', y='ECLO', hue='도로형태2', data=tr_df, errorbar=None)
# plt.legend(loc='upper right')
# plt.show()

In [ ]:
# tr_df.groupby(['도로형태1','도로형태2'])[['ECLO']].mean()

In [ ]:
# 도로형태2가 도로형태1의 내용을 포함하고 있음으로 제거함
# tr_df.drop(['도로형태1'],axis=1,inplace=True)
# tr_df.rename(columns={'도로형태2':'도로형태'},inplace=True)
# tr_df.columns

In [ ]:
col_name = {'요일':'week','기상상태':'weather','노면상태':'surface',
    '사고유형':'how_accident','사고유형 - 세부분류':'accident_detail',
    '법규위반':'nolaw','가해운전자 연령':'perpetrator_age',
    '가해운전자 상해정도':'perpetrator_injury','피해운전자 차종':'victim_car',
    '가해운전자 차종':'perpetrator_car','가해운전자 성별':'perpetrator_sex',
    '피해운전자 성별':'victim_sex','피해운전자 연령':'victim_age',
    '피해운전자 상해정도':'victim_injury','사망자수':'dead',
    '중상자수':'seriously_injured','경상자수':'minor_casualty',
    '부상자수':'injured','연':'year','월':'month','일':'day','시간':'hour',
    '구':'gu','동':'dong','도로형태':'road','보안등 설치개수':'light',
    'CCTV설치대수':'CCTV','제한속도':'limitspeed',
    '횡단보도개수':'cross_count','통행량':'public_traffic'}

In [ ]:
tr_df.rename(columns=col_name,inplace=True)
tr_df.columns

In [ ]:
tr_df['accident_detail'].value_counts()

In [ ]:
plt.figure(figsize=(20,5))
sns.scatterplot(data = tr_df, x = 'accident_detail', y = 'ECLO')

In [ ]:
plt.figure(figsize=(20,5))
sns.scatterplot(data = tr_df, x = 'light', y = 'ECLO')

## test

In [ ]:
te_df.isnull().sum()

In [ ]:
te_df.rename(columns={'설치개수':'보안등 설치개수'},inplace=True)

In [ ]:
te_df['CCTV설치대수'] = te_df['CCTV설치대수'].fillna(te_df['CCTV설치대수'].mean())
te_df['보안등 설치개수'] = te_df['보안등 설치개수'].interpolate()
te_df['제한속도'] = te_df['제한속도'].interpolate()

In [ ]:
te_df['횡단보도개수'] = te_df['횡단보도개수'].interpolate()
te_df['통행량'] = te_df['통행량'].interpolate()

In [ ]:
te_df.isnull().sum()

In [ ]:
# 도로형태2가 도로형태1의 내용을 포함하고 있음으로 제거함
# te_df.drop(['도로형태1'],axis=1,inplace=True)
# te_df.rename(columns={'도로형태2':'도로형태'},inplace=True)
# te_df.columns

In [ ]:
col_name = {'요일':'week','기상상태':'weather','노면상태':'surface','사고유형':'how_accident',
                     '연':'year','월':'month','일':'day','시간':'hour','구':'gu','동':'dong','도로형태':'road','보안등 설치개수':'light','CCTV설치대수':'CCTV','제한속도':'limitspeed',
           '횡단보도개수':'cross_count','통행량':'public_traffic'}

In [ ]:
te_df.rename(columns=col_name,inplace=True)
te_df.columns

# 인코딩

In [ ]:
pip install category_encoders

In [ ]:
from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder

In [ ]:
te_df.columns
#카테고리 열이 아니면 그래프 그려서 분포 확인해보기

In [ ]:
train_x = tr_df[te_df.columns].copy()
train_y = tr_df['ECLO'].copy()

In [ ]:
categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
categorical_features

In [ ]:
categorical_features_test = list(te_df.dtypes[te_df.dtypes == "object"].index)
categorical_features_test

In [ ]:
numeric_features_test = list(te_df.dtypes[te_df.dtypes == "int"].index)
numeric_features_test.extend(list(te_df.dtypes[te_df.dtypes == "float"].index))

In [ ]:
numeric_features_test

In [ ]:
plt.hist(train_x['light'])
plt.title('light')
plt.show()

In [ ]:
plt.hist(train_x['CCTV'])
plt.title('CCTV')
plt.show()

In [ ]:
plt.hist(train_x['limitspeed'])
plt.title('limitspeed')
plt.show()

In [ ]:
train_x[train_x['limitspeed'] == train_x['limitspeed'].max()].head()

In [ ]:
train_x['light'].describe()

In [ ]:
te_df['light'].describe()

In [ ]:
def change_range(x):
  if x > train_x['light'].mean():
    return 1
  else:
    return 0

In [ ]:
train_x['light_range'] = train_x['light'].apply(change_range)

In [ ]:
train_x['light_range'].value_counts()

In [ ]:
def change_range_test(x):
  if x > te_df['light'].mean():
    return 1
  else:
    return 0

In [ ]:
te_df['light_range'] = te_df['light'].apply(change_range_test)
te_df['light_range'].value_counts()

In [ ]:
plt.hist(te_df['light'])
plt.title('light')
plt.show()

In [ ]:
plt.hist(te_df['CCTV'])
plt.title('CCTV')
plt.show()

In [ ]:
plt.hist(te_df['limitspeed'])
plt.title('limitspeed')
plt.show()

In [ ]:
#for i in categorical_features:
#    le = TargetEncoder(cols=[i])
#    train_x[i] = le.fit_transform(train_x[i], train_y)
#    te_df[i] = le.transform(te_df[i])

#display(train_x.head())
#display(te_df.head())

In [ ]:
categorical_features

In [ ]:
# Mean target encoding
Week_mean_target_encoded = tr_df.groupby('week')['ECLO'].mean()
Weather_mean_target_encoded = tr_df.groupby('weather')['ECLO'].mean()
Surface_mean_target_encoded = tr_df.groupby('surface')['ECLO'].mean()
How_accident_mean_target_encoded = tr_df.groupby('how_accident')['ECLO'].mean()
Gu_mean_target_encoded = tr_df.groupby('gu')['ECLO'].mean()
Dong_mean_target_encoded = tr_df.groupby('dong')['ECLO'].mean()
Road_mean_target_encoded = tr_df.groupby('road')['ECLO'].mean()

In [ ]:
train_x['week_encoded']=train_x['week'].map(Week_mean_target_encoded)
train_x['weather_encoded']=train_x['weather'].map(Weather_mean_target_encoded)
train_x['surface_encoded']=train_x['surface'].map(Surface_mean_target_encoded)
train_x['how_accident_encoded']=train_x['how_accident'].map(How_accident_mean_target_encoded)
train_x['gu_encoded']=train_x['gu'].map(Gu_mean_target_encoded)
train_x['dong_encoded']=train_x['dong'].map(Dong_mean_target_encoded)
train_x['road_encoded']=train_x['road'].map(Road_mean_target_encoded)

In [ ]:
train_x['weather'].value_counts()

In [ ]:
train_x['surface'].value_counts()

In [ ]:
train_x['how_accident'].value_counts()

In [ ]:
train_x['gu'].value_counts()

In [ ]:
train_x['dong'].value_counts()

In [ ]:
train_x['road'].value_counts()

In [ ]:
categorical_features_test

In [ ]:
numeric_features_test

In [ ]:
te_df['week_encoded']=te_df['week'].map(Week_mean_target_encoded)
te_df['weather_encoded']=te_df['weather'].map(Weather_mean_target_encoded)
te_df['surface_encoded']=te_df['surface'].map(Surface_mean_target_encoded)
te_df['how_accident_encoded']=te_df['how_accident'].map(How_accident_mean_target_encoded)
te_df['gu_encoded']=te_df['gu'].map(Gu_mean_target_encoded)
te_df['dong_encoded']=te_df['dong'].map(Dong_mean_target_encoded)
te_df['road_encoded']=te_df['road'].map(Road_mean_target_encoded)

데이터 불균형 해소 필요해 보임!(train에만 적용)
- SMOTE 이용

In [ ]:
display(train_x.columns)
display(te_df.columns)

In [ ]:
# train_df.corr()

In [ ]:
plt.hist(train_y)

In [ ]:
train_y_log = np.log1p(train_y)
sns.distplot(train_y_log)
plt.show()

# 모델링

In [ ]:
train_x_new = train_x[['week_encoded', 'weather_encoded', 'surface_encoded',
       'how_accident_encoded', 'gu_encoded', 'dong_encoded', 'road_encoded','year', 'month', 'day','hour','CCTV', 'limitspeed',
       'light_range','cross_count','public_traffic']]

In [ ]:
te_df_new = te_df[['week_encoded', 'weather_encoded', 'surface_encoded',
       'how_accident_encoded', 'gu_encoded', 'dong_encoded', 'road_encoded','year', 'month', 'day','hour','CCTV', 'limitspeed',
       'light_range','cross_count','public_traffic']]

In [ ]:
# 에러나서 보류
#from imblearn.over_sampling import SMOTE
#smote = SMOTE()
#train_X_over, train_Y_over = smote.fit_resample(train_x_new,train_y)

파이토치를 이용해 모델 구현

In [ ]:
#from sklearn.model_selection import GridsearchCV
#from sklearn.linear_model import LinearRegreesion,Lasso,Ridge
#from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler #📢 minmax로 수정
# scaler = StandardScaler()
scaler = MinMaxScaler()

normed_train_data = pd.DataFrame(scaler.fit_transform(train_x_new),columns=train_x_new.columns)
normed_test_data = pd.DataFrame(scaler.transform(te_df_new),columns=te_df_new.columns)

In [ ]:
'''
def rmse(y,pred):
  rmse = np.sqrt(mean_squared_error(y,pred))
  return rmse

def rmsle(y,pred):
  log_y = np.log1p(y)
  log_pred = np.log1p(pred)
  squared_error = (log_y - log_pred)**2
  rmsle = np.sqrt(np.mean(squared_error))
  return rmsle

def eval_reg(y,pred):
  rmse_val = rmse(y,pred)
  rmsle_val = rmsle(y,pred)
  mae = mean_absolute_error(y,pred)
  print('RMSLE : {0:3f} RMSE : {2:3f} MAE : {2:3f}',format(rmsle_val,rmse_val,mae))
'''

In [ ]:
# import tensorflow as tf

In [ ]:
# def rmsle(y_true, y_pred):
#     y_true = tf.cast(y_true, tf.float32)
#     y_pred = tf.cast(y_pred, tf.float32)

#     squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))

#     return tf.sqrt(tf.reduce_mean(squared_error))

# def loss_fn(y_true, y_pred):
#     return rmsle(y_true, y_pred)

# def metric_fn(y_true, y_pred):
#     return rmsle(y_true, y_pred)

In [ ]:
# callbacks_list = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
#     tf.keras.callbacks.TerminateOnNaN()
# ]

In [ ]:
# def create_model():

#     input_layer = tf.keras.Input(shape=(len(normed_train_data.columns), ))
#     x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
#     x = tf.keras.layers.Dense(16, activation='relu')(x)
#     #x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
#     x = tf.keras.layers.Dense(32, activation='relu')(x)
#     #x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
#     output_layer = tf.keras.layers.Dense(1)(x)

#     model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

#     model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
#                   loss=loss_fn,
#                   metrics=[metric_fn]
#                   )

#     return model

# model = create_model()
# history = model.fit(normed_train_data.astype('float32'), train_y_log.astype('float32'), #type 맞춰줘야함!!!!
#                     epochs=100,
#                     callbacks=callbacks_list,
#                     validation_split=0.1)

In [ ]:
# # 이미 float64형
# normed_train_data.info()

In [ ]:
# train_y_log

In [ ]:
# # train loss, val loss 시각화
# import matplotlib.pyplot as plt

# plt.plot(history.history['loss'], 'b-', label='loss')
# plt.plot(history.history['val_loss'], 'r--', label='val_loss')
# plt.xlabel('Epoch')
# plt.legend()
# plt.show()

In [ ]:
# sample_submission = pd.read_csv('/content/drive/Othercomputers/내 Mac/공모전 준비/sample_submission.csv')

# sample_submission["ECLO"] = np.expm1(model.predict(normed_test_data.astype('float32')))

# sample_submission.to_csv("/content/drive/Othercomputers/내 Mac/공모전 준비/submission_ver8.csv", index=False)

In [ ]:
# sample_submission

In [ ]:
#📢토치 모델
import copy

# Early stopping (see module 3.4)
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,y_pred,y_true):
        rmsle = torch.sqrt(self.mse(torch.log1p(y_pred),torch.log1p(y_true)))
        return rmsle
    
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.seq = nn.Sequential(nn.Linear(input_dim, 32, bias=True),
                                nn.BatchNorm1d(1),
                                nn.ReLU(),
                                nn.Linear(32,64),
#                                 nn.BatchNorm1d(1),
                                nn.Dropout(0.25),
                                nn.ReLU(),
                                nn.Linear(64,1))
    def forward(self, x):
        out = self.seq(x)
        return out
    
### random seed setting
torch.manual_seed(42)

In [ ]:
# GPU device
# GPU 확인
print(torch.cuda.is_available())
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(torch.cuda.get_device_name())
print(torch.cuda.device_count())

In [ ]:
# dataset
'''
normed_train_data, train_y_log
normed_test_data
'''

In [ ]:
# prepare tensor data
train_x_tensor = Variable(torch.Tensor(normed_train_data.values.astype('float64'))).unsqueeze(1).to(device)
train_y_tensor = Variable(torch.Tensor(train_y_log.astype('float64'))).unsqueeze(1).to(device)
test_x_tensor = Variable(torch.Tensor(normed_test_data.values.astype('float64'))).unsqueeze(1).to(device)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
patience = 50
fold = 0
for train_idx, val_idx in kf.split(train_x_tensor):
    fold += 1
    print(f'Fold #{fold}')
    x_train, x_val = train_x_tensor[train_idx], train_x_tensor[val_idx]
    y_train, y_val = train_y_tensor[train_idx], train_y_tensor[val_idx]
    
    # DataLoader
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    
    # Create model
    model = Model(train_x_tensor.size(2)).to(device) # = num_features
    
    # Optimizer, criterion
    optimizer = optim.Adam(model.parameters())
    loss_fn = RMSLELoss()
    
    # EarlyStopping
    best_loss = float('inf')
    early_stopping_counter = 0
    
    # Training loop
    EPOCHS = 500
    epoch = 0
    done = False
    es = EarlyStopping(patience=patience)
    
    while not done and epoch<EPOCHS:
        epoch += 1
        model.train() # set as train mode
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = loss_fn(output,y_batch)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval() # set as test mode
        with torch.no_grad():
            val_output = model(x_val)
            val_loss = loss_fn(val_output, y_val)
        if es(model, val_loss):
            done = True
    
    print(f"Epoch: {epoch}/{EPOCHS}, Validation Loss: {val_loss.item()}, {es.status}")

In [ ]:
# Predict
final_pred = model(test_x_tensor)

In [ ]:
# check output
final_pred_log = final_pred.detach().squeeze(1).to('cpu').numpy()
final_pred = np.expm1(final_pred_log)
print(final_pred.shape, final_pred[:10])

In [ ]:
sample = pd.read_csv(path+'sample_submission.csv')
sample['ECLO'] = final_pred
sample

In [ ]:
sample = pd.read_csv(path+'sample_submission.csv')
sample['ECLO'] = final_pred
sample

In [ ]:
# save
sample.to_csv("../data/submits/torch_basic_1204_001.csv",index=False)